In [71]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [72]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [73]:
train_data = train_data.drop('user_id', axis = 1)
test_data = test_data.drop('user_id', axis = 1)

In [74]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

In [75]:
x = train_data.drop('target', axis = 1)
y = train_data['target']

In [76]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)

In [77]:
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size=0.2 ,random_state = 42)

In [88]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16,32)
        self.fc2 = nn.Linear(32,64)
        self.fc3 = nn.Linear(64,1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [89]:
input_size = 17
model = MyModel()

In [90]:
bce = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [91]:
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [92]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

In [93]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [96]:
test_data = test_data.to_numpy()

In [97]:
test = torch.tensor(test_data, dtype = torch.float32)

In [94]:
epochs = 1000

# GPU 사용 가능 여부를 확인하고, GPU 또는 CPU에 모델을 옮깁니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 지정된 에폭 수만큼 모델을 학습합니다.
for epoch in range(epochs):
    model.train()  # 모델을 학습 모드로 설정합니다.
    running_loss = 0.0

    # 미니배치 별로 데이터를 사용하여 학습합니다.
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # 그래디언트 초기화
        outputs = model(inputs)  # 모델에 입력 데이터를 전달하여 출력을 얻습니다.
        loss = bce(outputs.squeeze(), labels)  # 예측값과 실제값의 차이를 계산하여 손실을 구합니다.
        loss.backward()  # 역전파 수행
        optimizer.step()  # 옵티마이저로 모델 파라미터 업데이트

        running_loss += loss.item() * inputs.size(0)  # 러닝 손실 계산

    epoch_loss = running_loss / len(train_loader.dataset)  # 에폭별 평균 손실 계산
    print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {epoch_loss:.4f}")

# 모델 평가
model.eval()  # 모델을 평가 모드로 설정합니다.
with torch.no_grad():
    x_val_tensor, y_val_tensor = x_val_tensor.to(device), y_val_tensor.to(device)
    outputs = model(x_val_tensor)  # 검증 데이터로 모델의 예측값을 계산합니다.
    predicted = (outputs.cpu().numpy() > 0.5).astype(int)  # 출력값을 기준으로 이진 분류를 수행합니다.
    accuracy = (predicted == y_val).mean()  # 정확도 계산
    print(f"Validation Accuracy: {accuracy:.4f}")

Epoch [1/1000] - Loss: 0.6627
Epoch [2/1000] - Loss: 0.6528
Epoch [3/1000] - Loss: 0.6506
Epoch [4/1000] - Loss: 0.6483
Epoch [5/1000] - Loss: 0.6458
Epoch [6/1000] - Loss: 0.6440
Epoch [7/1000] - Loss: 0.6424
Epoch [8/1000] - Loss: 0.6405
Epoch [9/1000] - Loss: 0.6384
Epoch [10/1000] - Loss: 0.6366
Epoch [11/1000] - Loss: 0.6352
Epoch [12/1000] - Loss: 0.6334
Epoch [13/1000] - Loss: 0.6315
Epoch [14/1000] - Loss: 0.6303
Epoch [15/1000] - Loss: 0.6271
Epoch [16/1000] - Loss: 0.6255
Epoch [17/1000] - Loss: 0.6238
Epoch [18/1000] - Loss: 0.6211
Epoch [19/1000] - Loss: 0.6198
Epoch [20/1000] - Loss: 0.6170
Epoch [21/1000] - Loss: 0.6163
Epoch [22/1000] - Loss: 0.6138
Epoch [23/1000] - Loss: 0.6119
Epoch [24/1000] - Loss: 0.6102
Epoch [25/1000] - Loss: 0.6077
Epoch [26/1000] - Loss: 0.6069
Epoch [27/1000] - Loss: 0.6060
Epoch [28/1000] - Loss: 0.6028
Epoch [29/1000] - Loss: 0.6011
Epoch [30/1000] - Loss: 0.6007
Epoch [31/1000] - Loss: 0.5993
Epoch [32/1000] - Loss: 0.5963
Epoch [33/1000] -

KeyboardInterrupt: 

In [98]:
# 모델을 사용하여 예측값 계산 (테스트 데이터에 대해)
model.eval()  # 모델을 평가 모드로 설정합니다.
with torch.no_grad():
    x_test_tensor = torch.tensor(test, dtype=torch.float32).to(device)
    outputs = model(x_test_tensor)  # 테스트 데이터에 대한 모델의 예측값을 계산합니다.
    y_pred = (outputs.cpu().numpy() > 0.5).astype(int)  # 이진 분류를 위해 임계값을 기준으로 예측값 생성


C:\Users\ccg\AppData\Local\Temp\ipykernel_14332\3363399831.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_tensor = torch.tensor(test, dtype=torch.float32).to(device)


In [99]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])